## Naive Bayes Classifier

In this lab we will study our very first machine learning algorithm, the Naive Bayes Classifier.  The lab is divided into two parts. <br>
In part 1, you will get to implement the Naive Bayes Classifier from scratch without using any libraries.  This will help you understand the principles behind the algorithm. <br>
In part 2, you will be guided through an end-to-end machine learning workflow, from data preprocessing to model fitting and model selection.  This can serve as a useful template for you to carry out any machine learning project.  

### Part I: Building Naive Bayes Classifier from scratch

Let's first review a fundamental theorem in probability, the Bayes Theorem.  This will form an important basis to develop our algorithm <br>
For any two events $A$ and $B$,

$$
\Pr(A,B) = \Pr(A|B) \times \Pr(B) \\
\Pr(B,A) = \Pr(B|A) \times \Pr(A)
$$

Since the LHS are the same, we have

$$
\Pr(A|B) \times \Pr(B) = \Pr(B|A) \times \Pr(A)
$$

Rearranging gives us the Bayes theorem

$$
\Pr(c|D) = \frac{\Pr(c) \times \Pr(D|c)}{\Pr(D)}
$$

Keep this theorem in mind as we will refer to this soon

Throughout part I of the lab we will be using a weather prediction problem as a guiding example.  The problem is a classification problem where we try to predict whether it would rain or not, i.e. two classes.<br> 

In order to develop a model, we first need to build a training dataset.  We do so by collecting some data over a month.  In the morning we record whether it is windy or not and whether it is cloudy or not.  We also have a target variable column *rainy* indicating whether it rains or not in the morning.  Collected data is stored in *weather.csv*, let's take a peek as below.

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("weather.csv")

#### Prior belief

Now we have the dataset loaded, lets try some prediction.  Before you leave for school in the morning you want to predict whether it will rain in the next few hours.  You have not left your flat yet.  What *prior* belief do you have for the probability that it rains?
<br><i>(Recall what is prior probability)</i>

In [2]:
df.head() # Here we can see some of the days (index) and their respective weather conditions.

,windy,cloudy,rainy
0,1,1,1
1,0,1,1
2,1,1,0
3,1,1,1
4,0,1,0


In [3]:
# Let's calculate the prior probability that it rains.
prio_pr_rain = len(df[df.rainy==1]) / len(df) # P(Rain) = The number of days it rains / The total number of days.
print("Prior probability that it rains:", round(prio_pr_rain,5))
# print("Number of days it rained: ", len(df[df.rainy==1]))
# print("Total number of days: ", len(df))

Prior probability that it rains: 0.4375


#### Posterior probability

You then leave your flat and see a dark cloud forming.  Would that modify your *belief*?  Very likely you believe that probability of rain is going to increase. <br>

But how do we quantify that?  In fact what we are interested in is the probability that it will rain given that it is cloudy.  Formally in notation we want

$$
\Pr(rainy | cloudy)
$$

Applying the Bayes theorem will get us

$$
\Pr(rainy|cloudy) = \frac{\Pr(rainy) \times \Pr(cloudy|rainy)}{\Pr(cloudy)}
$$

<hr>
To get: 
Pr(cloudy|rainy)

$$
\Pr(cloudy|rainy) = \frac{\Pr(cloudy,rainy)}{\Pr(rainy)}
$$

In [4]:
# To find the probability of rain given that it is cloudy, we can use the formula:
# P(Cloudy|Rain) = P(Rain n Cloudy) / P(Rain)
pr_cloudy_given_rainy = len(df[(df.cloudy==1) & (df.rainy==1)])  /  len(df[df.rainy==1])

# Hence, using Bayes' Theorem, we can find the probability that it rains given that it is cloudy.
pr_cloudy = len(df[df.cloudy==1])  /  len(df) # P(Cloudy) is the total number of rows in df that has 1 in the 'Cloudy' Attribute/Class.
pr_rain = prio_pr_rain * pr_cloudy_given_rainy / pr_cloudy # P(Rain|Cloudy) = [P(Rain) * P(Cloudy|Rain)] / P(Cloudy)
print("Probability that it rains given that it is cloudy:", round(pr_rain,5))

Probability that it rains given that it is cloudy: 0.70588


Further explaining `pr_rain`:
$$
\Pr(rainy|cloudy) = \frac{\Pr(cloudy, rainy)}{\Pr(cloudy)}
$$

$$
\Pr(cloudy, rainy) = \Pr(cloudy|rainy) \times \Pr(rainy)
$$

Therefore, 

$$
\Pr(rainy|cloudy) = \frac{\Pr(cloudy|rainy) \times \Pr(rainy)}{\Pr(cloudy)}
$$

This is an example of Bayesian thinking, where we don't fix the probability of an event happening.  In fact we as we gather more *data*, we will *update* our beliefs.  Contrast it to the classical school where the probability of an event happening is expressed in long-term frequency. <br>

You then move along the street and suddenly feel strong wind blowing.  How do we further update our beliefs?  To start, we first need to define what the quantity we interested.  In words, it is the probability of rain *given* that it is cloudy *and* windy.  Notationally we want to calculate

$$
\Pr(rainy | cloudy, windy)
$$

Applying again the Bayes theorem will now get us the expression below. <br>
Note that to make the expressions more readable we use $R$ to represent rainy, $C$ to represent cloudy and $W$ to represent windy

$$
\Pr(R|C,W) = \frac{\Pr(R) \times \Pr(C,W | R)}{\Pr(C,W)}
$$

Here we will make an assumption to simplify the calculation.  We will assume that the cloudy events are independent of the windy events, conditioned on rainy events.  And this is the reason why it is called *Naive* Bayes.  We also drop denominator and write

$$
\Pr(R|C,W) \propto \Pr(R) \times \Pr(C | R) \times \Pr(W | R) \tag{1}
$$

Similarly lets write down the probability of not rainy given the same data - cloudy and windy

$$
\Pr(\sim R|C,W) \propto \Pr(\sim R) \times \Pr(C | \sim R) \times \Pr(W | \sim R) \tag{2}
$$

Finally we can simply compare the RHS of equation (1) and (2).  If the quantity from (1) is larger then the given cloud, windy, the probability of rain is higher than that of not rainy.  To express the RHS of (1) and (2) as probabililities, we could *normalize* them as shown below

In [5]:
pr_rainy = len(df[df.rainy==1]) / len(df) # P(Rain)
pr_cloudy_given_rainy = len(df[(df.rainy==1) & (df.cloudy==1)]) / len(df[df.rainy==1]) # P(Cloudy|Rain) = P(Rain n Cloudy) / P(Rain)
pr_windy_given_rainy = len(df[(df.rainy==1) & (df.windy==1)]) / len(df[df.rainy==1]) # P(Windy|Rain) = P(Rain n Windy) / P(Rain)

pr_not_rainy = len(df[df.rainy==0]) / len(df) # P(Not Rain) or P(Rain')
pr_cloudy_given_not_rainy = len(df[(df.rainy==0) & (df.cloudy==1)]) / len(df[df.rainy==0]) # P(Cloudy|Not Rain) = P(Not Rain n Cloudy) / P(Not Rain)
pr_windy_given_not_rainy = len(df[(df.rainy==0) & (df.windy==1)]) / len(df[df.rainy==0]) # P(Windy|Not Rain) = P(Not Rain n Windy) / P(Not Rain)

# Here, given that it is cloudy and windy, we can calculate the probability that it rains.
post_pr_rainy = pr_rainy * pr_cloudy_given_rainy * pr_windy_given_rainy # P(Rain|Cloudy n Windy) = P(Rain) * P(Cloudy|Rain) * P(Windy|Rain)

# Likewise, we can calculate the probability that it does not rain given that it is cloudy and windy.
post_pr_not_rainy = pr_not_rainy * pr_cloudy_given_not_rainy * pr_windy_given_not_rainy # P(Not Rain|Cloudy n Windy) = P(Not Rain) * P(Cloudy|Not Rain) * P(Windy|Not Rain)

# The sum of the probabilities that it rains and that it does not rain given that it is cloudy and windy, which is equal to the probability that it is cloudy and windy.
p_sum = post_pr_rainy + post_pr_not_rainy # P(Rain|Cloudy n Windy) + P(Not Rain|Cloudy n Windy) = P(Cloudy n Windy).

# Normalize the probabilities to find the probability that it rains given that it is cloudy and windy.
post_pr_rainy / p_sum

0.6492985971943888

And we just implemented Naive Bayes from scratch, even though it is only for two features.  In Exercise 1, you will extend this to write a generic function to predict a class based on all feature values.

##### Further Explaination:

Let's related back to the document classifier we learnt in class earlier: <br>
Here, $R$ is is our class as that is what we are looking for, and $W$ and $C$ are the Tokens. <br>
$W$ and $C$ can also be classes, given their own scenarios, but for now we only focus on $R$.

If you take a good look at this formula again, <br>
$$
\Pr(R|C,W) = \frac{\Pr(R) \times \Pr(C,W | R)}{\Pr(C,W)} $$
$$
\Pr(c|d5) = \frac{\Pr(c) \times \Pr(d5 | c)}{\Pr(d5)}
$$
Both of these formulas are similar.

So in this case, our $d5$ is $C,W$ or the intersection between $C$ and $W$.

Now that we make the correlation, we know what to do. <br>
Remember during the document classification, we calculated that <br>
$$
\Pr(c|d5) \propto \frac{\Pr(d5|c) \times \Pr(c)}{\Pr(d5)} = \Pr(Chinese1|c) \times \Pr(Chinese2|c) \times \Pr(Chinese3|c) \times \Pr(Tokyo|c) \times \Pr(Japan|c) \times \Pr(c)
$$

Now, applying it onto this problem, we find that we already have something similar: <br>
$$
\Pr(R|C,W) \propto \Pr(R) \times \Pr(C | R) \times \Pr(W | R)
$$

Hence, our final result can be `post_pr_rainy`. <br>
However, that is only finding the probability for one class, and how the total probability of raining.<br>
Thus, we will need to normalize.

We can do normalization by dividing the class probability against the total probability of Cloudy and Windy. <br>
Which is shown through `post_pr_rainy` / `p_sum`.

Why is `p_sum` the result of combining all `post_pr_rainy` and `post_pr_not_rainy`? <br>
That is because given that it is Cloudy and Windy, there are only 2 types of outcomes: Rainy or Not Rainy. Thus, adding both together gives you the sum total of Cloudy and Windy.

#### Exercise 1: Naive Bayes implementation

Write a function *naive_bayes_predict* which takes in two parameters and returns the predicted class.  The first input parameter is the dataframe containing feature columns and a target column.  The second input parameter is a dictionary with features and their corresponding values are key-value pairs.  And you are supposed to predict the class for this instance.<br>
For example, given the following, your function will predict the class for instance where *feature_1*=1, *feature_2*=1, *feature_3=0* and *feature_4=1*.  You may assume that the last column of the input dataframe is called *target*, but in general there can be any number of columns.  Also, all values in the dataframe can be assumed to be either 1 or 0.

In [6]:
train = pd.read_csv("train.csv")
test = {"feature_1":1, "feature_2":1, "feature_3":0, "feature_4":1}

train.head()

#naive_bayes_predict(train, test) ---> output: 1 or 0

,feature_1,feature_2,feature_3,feature_4,target
0,1,1,0,1,1
1,0,1,0,1,1
2,1,1,1,0,0
3,1,1,0,0,1
4,0,1,1,0,0


##### Answer:

In [7]:
def naive_bayes_predict(train , test):
    df = train
    
    # Prior Probability of the 'True' Class: target = 1.
    pr_true = len(df[df.target==1]) / len(df)
    # Finding the probability of each feature in the test set being in the 'True' Class
    for key, value in test.items(): # For each feature in the test set,
        pr_true *= ( len(df[(df.target==1) & (df[key]==value)]) / len(df[df.target==1]) )
        # Breakdown:
            # len(df[(df.target==1) & (df[key]==value)] Finds the number of rows in df that are both in the 'True' Class, 
            # and has a feature value that correspondingly matches the current test set's feature value.
            # Dividing the frequency of the above by len(df[df.target==1] or the total number of rows that in df that are
            # part of the 'True' Class finds the probability of the 'True' Class.
    
    # Prior Probability of the 'False' Class: target = 0.
    pr_false = len(df[df.target==0]) / len(df)
    # Finding the probability of each feature in the test set being in the 'True' Class
    for key, value in test.items(): # For each feature in the test set,
        pr_false *= ( len(df[(df.target==0) & (df[key]==value)]) / len(df[df.target==0]) )
        # Breakdown:
            # len(df[(df.target==1) & (df[key]==value)] Finds the number of rows in df that are both in the 'True' Class, 
            # and has a feature value that correspondingly matches the current test set's feature value.
            # Dividing the frequency of the above by len(df[df.target==1] or the total number of rows that in df that are
            # part of the 'True' Class finds the probability of the 'True' Class.

    print(pr_true)
    print(pr_false)
    print(pr_true/(pr_true+pr_false)) # Proportion of True in the test set.
    return 1 if pr_true>pr_false else 0 # If the Probability of 'True' is more than 'False', return 1. Else return 0.

print(naive_bayes_predict(train, test))

0.06641763848396502
0.017146776406035662
0.7948076770643738
1


### Part II: End to end ML workflow

In this part of the lab, we will walk through a simplified ML workflow.  Obviously an actual project will be far more complicated but this exercise will give you a good glimpse into major components of a ML project.  We will again use the *iris* dataset and use the Naive Bayes classifier from scikit-learn.

In [8]:
import numpy as np
import pandas as pd

df = pd.read_csv("iris.csv")

#### Step 1: Exploratory data analysis

The very first step involves developing a sense of what data you are dealing with.  Are they all continuous data?  Categorical data?  What about the distribution of the data?  Are there outliers?  Is the dataset balanced?  

In [9]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,variety
0,57.0,28.0,45.0,13.0,Versicolor
1,5.7,2.6,3.5,1.0,Versicolor
2,6.0,3.4,4.5,1.6,Versicolor
3,5.0,2.0,3.5,1.0,Versicolor
4,5.6,2.8,4.9,2.0,Virginica


In [10]:
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,6.185333,3.225333,4.028000,1.277333
std,4.258121,2.082435,3.801927,1.228583
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.375000,5.100000,1.800000
max,57.000000,28.000000,45.000000,13.000000


#### Step 2: Data peprocessing

Based on your EDA, you will need to preprocess the data.  If data is missing, you will have to impute.  For outliers, do you want to remove or replace the values?  Do we need to bin the data?  For example, in the given dataset, we noted that the first row seems like an outlier - this suggest we might want to limit the data to less than 10.

In [11]:
# Only keep rows that have features smaller than 10
df = df[ (df.sepal_length<10) & (df.sepal_width<10) & (df.petal_length<10) & (df.petal_width<10)]
df.head()

,sepal_length,sepal_width,petal_length,petal_width,variety
1,5.7,2.6,3.5,1.0,Versicolor
2,6.0,3.4,4.5,1.6,Versicolor
3,5.0,2.0,3.5,1.0,Versicolor
4,5.6,2.8,4.9,2.0,Virginica
5,5.7,4.4,1.5,0.4,Setosa


#### Step 3: Feature engineering

One of the most important steps in the ML workflow for structured data is manual feature engineering.  (Deep learning on the other hand learns feature engineering on its own).  This step typically comes from a specific domain knowledge.  A good set of features can beat a good algorithm sometimes.  In our example, we might want to create *average_length* and *average_width* as additional features

In [12]:
df["ave_length"] = (df.sepal_length + df.petal_length)/2
df["ave_width"] = (df.sepal_width + df.petal_width)/2
df.head()

,sepal_length,sepal_width,petal_length,petal_width,variety,ave_length,ave_width
1,5.7,2.6,3.5,1.0,Versicolor,4.60,1.8
2,6.0,3.4,4.5,1.6,Versicolor,5.25,2.5
3,5.0,2.0,3.5,1.0,Versicolor,4.25,1.5
4,5.6,2.8,4.9,2.0,Virginica,5.25,2.4
5,5.7,4.4,1.5,0.4,Setosa,3.60,2.4


#### Step 4: Split data into train and testing set

Before fitting our model, we need to split the dataset into *train* and *test* set.  The *train* dataset will be used to fit a model whereas the *test* set will help decide you between the different models or different hyperparameters.  To do the split, we can simply calll the *train_test_split* function from scikit-learn

In [13]:
from sklearn.model_selection import train_test_split
# training and test data set should have similar distribution

X = df.drop(columns=["variety"])
y = df["variety"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 80% training and 20% test

#### Step 5: Model fitting

Now we are ready to fit a model to our data.  We will again use scikit-learn and specifically the Gaussian Naive Bayes model.  The typical way to do this is to create an instance of the model class and call the *fit* method

In [14]:
from sklearn.naive_bayes import GaussianNB

clf1 = GaussianNB()
# priors - default prior probabilities of the classes, var_smoothing - https://stackoverflow.com/questions/58046129/can-someone-give-a-good-math-stats-explanation-as-to-what-the-parameter-var-smoo
# build a few models here and compare performance metrics

clf1.fit(X_train, y_train) # Applying the training data on the model

GaussianNB()

#### Step 6: Model validation

If we could justify the use of Gaussian Naive Bayes, then we are done!  However, more often than not, we will compare a few other models in order to select the right one.  Two things to note.  Firstly, we need to decide a metric to evaluate how good a model is.  Here we use the accuracy score.  Secondly, we need to use a different (unseen) dataset, the *validation* set to evaluate

In [15]:
from sklearn.metrics import accuracy_score

y_pred1 = clf1.predict(X_test) # Once the model has been trainined on the training data, we the apply the model on the testing data to get the desired predictions.

acc = accuracy_score(y_test, y_pred1) # Accuracy Score of the model against the actual test data
print("accuracy score:", round(acc,5))

accuracy score: 0.93333


In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred1)) # Confusion matrix: precision, recall, f1-score, support

              precision    recall  f1-score   support

      Setosa       1.00      1.00      1.00        10
  Versicolor       1.00      0.78      0.88         9
   Virginica       0.85      1.00      0.92        11

    accuracy                           0.93        30
   macro avg       0.95      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30



Can we do better?  How does it compare to Multinomial Naive Bayes?  You will find out in Exercise 2

#### Exercise 2: Multinomial Naive Bayes

In this exercise you will train the model using the Multinomial Naive Bayes.  See the  scikit-learn documentation <a href="https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html"> here </a>.  You will use the same train and test set, but may adjust accordingly if needed.  Compute the metric *accuracy_score* and provide a comment

##### Answer:

In [17]:
from sklearn.naive_bayes import MultinomialNB

clf2 = MultinomialNB() # Create the model
clf2.fit(X_train, y_train) # Applying the training data on the model

y_pred2 = clf2.predict(X_test) # Find the desired predicted value from by applying the testing data
acc = accuracy_score(y_test, y_pred2) # Compare its accuracy

print("accuracy score:", round(acc,5))

accuracy score: 0.93333


In [18]:
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

      Setosa       1.00      1.00      1.00        10
  Versicolor       0.82      1.00      0.90         9
   Virginica       1.00      0.82      0.90        11

    accuracy                           0.93        30
   macro avg       0.94      0.94      0.93        30
weighted avg       0.95      0.93      0.93        30



In [19]:
"""
- Bernoulli Naive Bayes for binary data

- Multinomial Naive Bayes for categorical data

- Gaussian Naive Bayes for continuous features with Gaussian distribution (Normal distribution)

all handles zero problem
"""

'\n- Bernoulli Naive Bayes for binary data\n\n- Multinomial Naive Bayes for categorical data\n\n- Gaussian Naive Bayes for continuous features with Gaussian distribution (Normal distribution)\n\nall handles zero problem\n'